# Data Ingestion

**Purpose**:

This notebook aims to conduct a lightweight data processing test using Python and SQL on an SQLite backend. We will adopt the medallion architecture, organising our data into Bronze, Silver, and Gold layers. Each layer will serve a specific purpose, beginning with the Bronze layer, which focuses on ingesting raw data.

Our data source is KaggleHub, and the dataset used is GoodReadBooks. In addition to this dataset, we will generate synthetic data to fill in gaps such as member information, books borrowed, return times, and other relevant details.

Finally, we will create views for the Gold layer to support refined and analytical use cases.

## Import 
We begin by importing the necessary libraries and functions required for data ingestion and processing.

In [23]:

from src.data.dataset_utils import create_sqlite_dataset, download_dataset_from_kagglehub, generate_member, generate_borrowing_records
import pandas as pd

## Dataset

The dataset will be downloaded from KaggleHub (jealousleopard/goodreadsbooks) and stored in the .cache directory for subsequent access and processing.

In [24]:


path = download_dataset_from_kagglehub("jealousleopard/goodreadsbooks")
path

Path to dataset files: /home/anish/.cache/kagglehub/datasets/jealousleopard/goodreadsbooks/versions/2


'/home/anish/.cache/kagglehub/datasets/jealousleopard/goodreadsbooks/versions/2'

## SQLite dataset
Next, we will create a database using the SQLite backend to store and manage the dataset for subsequent processing.

In [25]:
from src.data.dataset_utils import delete_sqlit_database
delete_sqlit_database("library.db") #we delete any exiting dataset called library.db for clean start

Database file does not exist.


In [26]:

conn, cursor = create_sqlite_dataset("library.db")

## Bronze layer table

Next, we will load the dataset. The GoodReadBooks dataset contains a books.csv file, which we will read using Pandas. This data will then be stored in our SQLite database as the bronze_books table, representing the raw data ingestion layer.

In [27]:
bronze_books = pd.read_csv(
    f"{path}/books.csv",
    on_bad_lines='skip',    
    quotechar='"',
    sep=","
)
display(bronze_books.sample(5))

bronze_books.to_sql("bronze_books", conn, if_exists="replace", index=False)


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
5505,19911,Captive Dreams,Angela Knight/Diane Whiteside,3.91,0425207757,9780425207758,en-US,336,1354,60,9/5/2006,Berkley Trade
10549,43015,A Long Way Gone: Memoirs of a Boy Soldier,Ishmael Beah,4.16,0374105235,9780374105235,eng,229,147820,9547,2/13/2007,Sarah Crichton Books
3802,13632,Museum of Terror Vol. 3: The Long Hair in the...,Junji Ito/伊藤潤二,4.12,1593076398,9781593076399,eng,392,628,26,11/21/2006,Dark Horse Manga
5368,19390,Philosophical Dictionary,Voltaire/Theodore Besterman,4.09,014044257X,9780140442571,en-US,400,1394,45,9/27/1979,Penguin Classics
9942,40051,Evening,Susan Minot,3.37,0375700269,9780375700262,eng,288,3765,550,9/7/1999,Vintage


11123

## Silver Layer Table

We will now create a cleaned version of the `bronze_books` table by performing the following tasks:

1. Create a `silver_books` table.
2. Filter out incomplete or invalid rows from the `bronze_books` table.
3. Remove any leading or trailing whitespace from book titles and author names.
4. Eliminate duplicate rows to ensure data consistency.

In [28]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS silver_books AS
SELECT DISTINCT
    bookID AS BookID,
    TRIM(title) AS Title,
    TRIM(authors) AS Author,
    publication_date AS YearPublished,
    average_rating AS AvgRating
FROM bronze_books
WHERE title IS NOT NULL
  AND authors IS NOT NULL
  AND publication_date IS NOT NULL
  AND average_rating IS NOT NULL;
""")
conn.commit()


Let’s preview five sample rows from the silver_books table to verify the data cleaning process.

In [29]:
pd.read_sql_query("SELECT * FROM silver_books LIMIT 5;", conn)


,BookID,Title,Author,YearPublished,AvgRating
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,9/16/2006,4.57
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,9/1/2004,4.49
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,11/1/2003,4.42
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,5/1/2004,4.56
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,9/13/2004,4.78


### Synthetic Data Generation for Members

We will now generate synthetic data to represent library members and their book borrowing behavior, as this information is not included in the original dataset.

#### Members table
We will create a members table to store synthetic data representing individual library members, which will include relevant attributes such as member ID, name, and registration details.

In [30]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Members (
    MemberID 
        INTEGER 
        PRIMARY KEY 
        AUTOINCREMENT,
    Name TEXT NOT NULL,
    JoinDate DATE DEFAULT (DATE('now'))
);
""")
conn.commit()


We will populate the `members` table with 50 synthetic entries to simulate a realistic set of library users.

In [31]:
members_list = generate_member(50)
members_list

[('Ivy Wilson', datetime.date(2024, 11, 11)),
 ('John White', datetime.date(2024, 11, 23)),
 ('Frank Thomas', datetime.date(2024, 12, 1)),
 ('Diana Wilson', datetime.date(2025, 2, 22)),
 ('Hassan Brown', datetime.date(2024, 9, 27)),
 ('Diana Taylor', datetime.date(2024, 10, 6)),
 ('Hassan Smith', datetime.date(2025, 4, 15)),
 ('Diana White', datetime.date(2025, 1, 4)),
 ('Alice Walker', datetime.date(2025, 3, 6)),
 ('Charlie Walker', datetime.date(2024, 6, 17)),
 ('Frank Smith', datetime.date(2024, 10, 3)),
 ('Eve White', datetime.date(2024, 12, 22)),
 ('Bob Roberts', datetime.date(2024, 10, 17)),
 ('Frank Evans', datetime.date(2024, 10, 23)),
 ('Grace Roberts', datetime.date(2024, 8, 18)),
 ('Ivy Smith', datetime.date(2024, 8, 26)),
 ('Hassan Walker', datetime.date(2025, 2, 1)),
 ('Eve Wilson', datetime.date(2025, 2, 10)),
 ('Eve Taylor', datetime.date(2024, 11, 14)),
 ('Grace Roberts', datetime.date(2025, 2, 18)),
 ('Hassan Evans', datetime.date(2025, 2, 28)),
 ('Alice Walker', datet

Next, we will insert the generated list into the members table.

In [32]:
cursor.executemany(
    "INSERT INTO Members (Name, JoinDate) VALUES (?, ?);",
    members_list
)
conn.commit()

pd.read_sql_query("SELECT * FROM Members;", conn)

,MemberID,Name,JoinDate
0,1,Ivy Wilson,2024-11-11
1,2,John White,2024-11-23
2,3,Frank Thomas,2024-12-01
3,4,Diana Wilson,2025-02-22
4,5,Hassan Brown,2024-09-27
5,6,Diana Taylor,2024-10-06
6,7,Hassan Smith,2025-04-15
7,8,Diana White,2025-01-04
8,9,Alice Walker,2025-03-06
9,10,Charlie Walker,2024-06-17


### Borrowing Records Table

We will now create a `borrowing_records` table to capture synthetic borrowing activity. Each record will include the book ID, member ID, borrow date, and return date. A `NULL` value in the return date column will indicate that the book has not yet been returned.


In [33]:
book_ids = pd.read_sql_query("SELECT BookID FROM silver_books LIMIT 100;", conn)["BookID"].tolist()
member_ids = pd.read_sql_query("SELECT MemberID FROM Members;", conn)["MemberID"].tolist()

In [34]:
borrowing_records = generate_borrowing_records(book_ids, member_ids)
borrowing_records

[(5, 1, datetime.date(2025, 3, 26), datetime.date(2025, 4, 3)),
 (4, 2, datetime.date(2025, 3, 14), datetime.date(2025, 4, 3)),
 (135, 2, datetime.date(2025, 3, 30), datetime.date(2025, 4, 11)),
 (129, 3, datetime.date(2025, 4, 11), datetime.date(2025, 4, 17)),
 (45, 4, datetime.date(2025, 6, 1), None),
 (140, 5, datetime.date(2025, 6, 2), datetime.date(2025, 6, 14)),
 (137, 6, datetime.date(2025, 3, 24), datetime.date(2025, 4, 18)),
 (16, 6, datetime.date(2025, 3, 25), None),
 (106, 7, datetime.date(2025, 4, 23), datetime.date(2025, 5, 4)),
 (154, 7, datetime.date(2025, 3, 13), None),
 (105, 8, datetime.date(2025, 3, 27), datetime.date(2025, 4, 6)),
 (85, 8, datetime.date(2025, 4, 1), None),
 (27, 8, datetime.date(2025, 5, 28), None),
 (31, 9, datetime.date(2025, 4, 2), datetime.date(2025, 4, 25)),
 (4, 10, datetime.date(2025, 5, 9), None),
 (86, 10, datetime.date(2025, 5, 23), None),
 (8, 10, datetime.date(2025, 5, 12), datetime.date(2025, 5, 29)),
 (122, 11, datetime.date(2025, 5, 3

In [35]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS BorrowingRecords (
    RecordID 
        INTEGER 
        PRIMARY KEY 
        AUTOINCREMENT,
    BookID INTEGER,
    MemberID INTEGER,
    BorrowDate 
        DATE 
        DEFAULT (DATE('now')),
    ReturnDate DATE,
    FOREIGN KEY (BookID) 
        REFERENCES silver_books(BookID),
    FOREIGN KEY (MemberID) 
        REFERENCES Members(MemberID)
);
""")
conn.commit()


Next, we will insert the generated borrowing data into the borrowing_records table to complete our synthetic dataset.

In [36]:
cursor.executemany(
    """
    INSERT INTO BorrowingRecords (BookID, MemberID, BorrowDate, ReturnDate)
    VALUES (?, ?, ?, ?);
    """,
    borrowing_records
)
conn.commit()

# Preview
pd.read_sql_query("SELECT * FROM BorrowingRecords LIMIT 5;", conn)


,RecordID,BookID,MemberID,BorrowDate,ReturnDate
0,1,5,1,2025-03-26,2025-04-03
1,2,4,2,2025-03-14,2025-04-03
2,3,135,2,2025-03-30,2025-04-11
3,4,129,3,2025-04-11,2025-04-17
4,5,45,4,2025-06-01,None


## Gold Layer View

For the Gold layer, we will adopt a different approach by creating SQL views instead of physical tables. These views will include refined queries to support analytics and reporting use cases.

### Top Rated Books View

We will create a view to identify top-rated books. Any book with an average rating greater than 4.5, based on the `silver_books` table, will be included in this view.



In [37]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS gold_top_books AS
SELECT 
    Title, 
    Author, 
    YearPublished, 
    AvgRating
FROM silver_books
WHERE AvgRating >= 4.5
ORDER BY AvgRating DESC
LIMIT 20;
""")

conn.commit()


In [38]:
pd.read_sql_query("SELECT * FROM gold_top_books LIMIT 5;", conn)

,Title,Author,YearPublished,AvgRating
0,Comoediae 1: Acharenses/Equites/Nubes/Vespae/P...,Aristophanes/F.W. Hall/W.M. Geldart,2/22/1922,5.0
1,Willem de Kooning: Late Paintings,Julie Sylvester/David Sylvester,9/1/2006,5.0
2,Literature Circle Guide: Bridge to Terabithia:...,Tara MacCarthy,1/1/2002,5.0
3,Middlesex Borough (Images of America: New Jersey),Middlesex Borough Heritage Committee,3/17/2003,5.0
4,Zone of the Enders: The 2nd Runner Official St...,Tim Bogenn,3/6/2003,5.0


From the sample results, we can observe that several books have received a perfect 5.0 rating, indicating exceptional user feedback.

### Top Borrowed Books View

We will create a view named `gold_top_borrowed_books` to identify the most frequently borrowed books. This view will:

* Join the `borrowing_records` table with the `silver_books` table using the `book_id` field.
* Retrieve relevant metadata such as book title and author.
* Group the data by `book_id`, count the total borrowings for each book, and order the results in descending order of borrow count.
* Limit the output to the top 10 most borrowed books.


In [39]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS gold_top_borrowed_books AS
SELECT 
    book.Title, 
    book.Author, 
    COUNT(*) AS BorrowCount
FROM BorrowingRecords AS borrow
JOIN silver_books AS book 
    ON borrow.BookID = book.BookID
GROUP BY borrow.BookID
ORDER BY BorrowCount DESC
LIMIT 10;
""")
pd.read_sql_query("SELECT * FROM gold_top_borrowed_books;", conn)


,Title,Author,BorrowCount
0,The Heidi Chronicles: Uncommon Women and Other...,Wendy Wasserstein,5
1,Ruby Ann's Down Home Trailer Park Cookbook,Ruby Ann Boxcar,4
2,The Lord of the Rings: The Art of the Fellowsh...,Gary Russell,4
3,The Heidi Chronicles,Wendy Wasserstein,4
4,Anna Karenina,Leo Tolstoy/Richard Pevear/Larissa Volokhonsky...,3
5,eBay Business All-in-One Desk Reference for Du...,Marsha Collier,3
6,Heirs of General Practice,John McPhee,3
7,The Changeling (Daughters of England #15),Philippa Carr,3
8,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,3
9,Untouchable,Mulk Raj Anand/E.M. Forster,2


### Top Members by Number of Borrowings

We will create a view to identify the top 10 members based on borrowing frequency. This will involve:

* Joining the `borrowing_records` table with the `members` table using the `member_id` field.
* Grouping the results by `member_id` and aggregating the total number of borrowings per member.
* Ordering the results in descending order and limiting the output to the top 10 borrowers.



In [40]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS gold_top_members AS
SELECT 
     member.Name, 
     COUNT(*) AS TotalBorrows        
FROM BorrowingRecords AS borrow          
JOIN Members AS member 
     ON borrow.MemberID = member.MemberID                  
GROUP BY borrow.MemberID
ORDER BY TotalBorrows DESC               
LIMIT 10;
""")
pd.read_sql_query("SELECT * FROM gold_top_members;", conn)


,Name,TotalBorrows
0,Diana Wilson,3
1,Bob Wilson,3
2,Frank Roberts,3
3,Hassan Walker,3
4,Alice White,3
5,Alice Roberts,3
6,Eve Brown,3
7,Hassan Roberts,3
8,Diana Roberts,3
9,Diana White,3


### Books Not Yet Returned

We will create a view to list all books that have not yet been returned by members. In this view:

* Only records with a `NULL` return date will be included.
* We will calculate whether each book is overdue by checking if the borrow date exceeds 14 days from the current date.
* The result will include relevant book and member details along with an `is_overdue` flag.



In [41]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS gold_unreturned_books AS
SELECT 
    member.Name AS Borrower,
    book.Title,
    borrow.BorrowDate,
    JULIANDAY('now') - JULIANDAY(borrow.BorrowDate) AS DaysOut,
    CASE 
        WHEN JULIANDAY('now') - JULIANDAY(borrow.BorrowDate) > 14 THEN 'Yes'
        ELSE 'No'
    END AS IsOverdue
FROM BorrowingRecords AS borrow
JOIN Members AS member 
    ON borrow.MemberID = member.MemberID
JOIN silver_books AS book 
    ON borrow.BookID = book.BookID
WHERE borrow.ReturnDate IS NULL;
""")
pd.read_sql_query("SELECT * FROM gold_unreturned_books;", conn)


,Borrower,Title,BorrowDate,DaysOut,IsOverdue
0,Diana Wilson,Agile Web Development with Rails: A Pragmatic ...,2025-06-01,8.344508,No
1,Diana Taylor,The Hitchhiker's Guide to the Galaxy (Hitchhik...,2025-03-25,76.344508,Yes
2,Hassan Smith,CliffsNotes on Tolstoy's Anna Karenina,2025-03-13,88.344508,Yes
3,Diana White,The Heidi Chronicles,2025-04-01,69.344508,Yes
4,Diana White,Neither Here nor There: Travels in Europe,2025-05-28,12.344508,No
5,Charlie Walker,Harry Potter and the Chamber of Secrets (Harry...,2025-05-09,31.344508,Yes
6,Charlie Walker,The Heidi Chronicles: Uncommon Women and Other...,2025-05-23,17.344508,Yes
7,Frank Smith,The Power of One (The Power of One #1),2025-05-03,37.344508,Yes
8,Eve White,Ruby Ann's Down Home Trailer Park Cookbook,2025-05-03,37.344508,Yes
9,Frank Evans,The Lord of the Rings: The Art of the Fellowsh...,2025-03-21,80.344508,Yes


### Month-by-Month Borrowing Activity

We will create a view to analyse borrowing trends over time by aggregating the number of books borrowed each month. This will help us observe borrowing patterns and seasonal activity.


In [42]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS gold_monthly_borrowing AS
SELECT 
    strftime('%Y-%m', BorrowDate) AS Month,
    COUNT(*) AS BorrowCount
FROM BorrowingRecords
GROUP BY Month
ORDER BY Month ASC;
""")
pd.read_sql_query("SELECT * FROM gold_monthly_borrowing;", conn)


,Month,BorrowCount
0,2025-03,29
1,2025-04,30
2,2025-05,42
3,2025-06,5


## Data Pipeline

Now that the data structure and transformations are validated, we will create a data ingestion pipeline to automate the population of all three layers—Bronze, Silver, and Gold.



In [43]:
from src.etl.bronze_etl import BronzeETL
from src.etl.silver_etl import SilverETL
from src.etl.gold_etl import GoldETL

def run_bronze_etl():
    bronze = BronzeETL()
    bronze.download_data()
    bronze.load_raw_books()
    bronze.write_to_sqlite()

def run_silver_etl():
    silver = SilverETL()
    silver.transform_bronze_books()
    silver.create_members_table(total_members=20)
    silver.create_borrowing_records_table()
    silver.preview_tables()

def run_gold_etl():
    gold = GoldETL()
    gold.create_top_books_table()
    gold.create_top_borrowed_books_view()
    gold.create_unreturned_books_view()
    gold.create_monthly_borrowing_view()
    gold.preview_gold_outputs()

def main():
    print("Running Bronze ETL...")
    run_bronze_etl()

    print("\nRunning Silver ETL...")
    run_silver_etl()

    print("\nRunning Gold ETL...")
    run_gold_etl()


# main()